In [13]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import logging
import subprocess
import os

In [14]:
# Set up logging
logging.basicConfig(level=logging.INFO)

In [15]:
# Load the pre-trained SRGAN model from TensorFlow Hub
srgan_module = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")


INFO:absl:Fingerprint not found. Saved model loading will continue.
INFO:absl:path_and_singleprint metric could not be logged. Saved model loading will continue.


In [16]:
# Function to preprocess and upscale frames
def upscale_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to the model's input size
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
    frame = frame / 255.0  # Normalize pixel values

    # Ensure the input is of type tf.float32
    frame = tf.convert_to_tensor(frame, dtype=tf.float32)

    frame = tf.expand_dims(frame, axis=0)  # Add batch dimension
    upscaled_frame = srgan_module(frame, training=False)  # Use the pre-trained model for upscaling
    upscaled_frame = tf.squeeze(upscaled_frame, axis=0)  # Remove batch dimension
    upscaled_frame = tf.clip_by_value(upscaled_frame, 0, 1)  # Clip values to [0, 1]
    upscaled_frame = tf.image.convert_image_dtype(upscaled_frame, dtype=tf.uint8)  # Convert to uint8
    upscaled_frame = tf.image.encode_png(upscaled_frame)  # Encode as PNG
    upscaled_frame = tf.image.decode_png(upscaled_frame)  # Decode from PNG
    upscaled_frame = tf.image.convert_image_dtype(upscaled_frame, dtype=tf.uint8)  # Convert to uint8
    return upscaled_frame.numpy()


In [17]:
# Function to process a video file
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    logging.info(f"Total frames in the video: {total_frames}")

    temp_folder = 'temp_frames'
    os.makedirs(temp_folder, exist_ok=True)

    frame_count = 0
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                logging.info("End of video reached.")
                break

            logging.info(f"Processing frame {frame_count}/{total_frames}")

            try:
                upscaled_frame = upscale_frame(frame)

                # Denoise the frame
                upscaled_frame = cv2.fastNlMeansDenoisingColored(
                    upscaled_frame,
                    None,  # Use default denoising parameters
                    h=5,    # Filter strength
                    hColor=5
                )

                # Save the upscaled and denoised frame to a temporary file
                temp_frame_path = os.path.join(temp_folder, f"frame_{frame_count:04d}.png")
                cv2.imwrite(temp_frame_path, cv2.cvtColor(upscaled_frame, cv2.COLOR_RGB2BGR))

            except Exception as e:
                logging.error(f"Error in processing frame: {e}")
                break

            frame_count += 1

        # Use ffmpeg to concatenate frames into a video
        ffmpeg_cmd = (
            f"ffmpeg -framerate {fps} -i {temp_folder}/frame_%04d.png "
            f"-c:v libx264 -pix_fmt yuv420p {output_path}"
        )
        subprocess.run(ffmpeg_cmd, shell=True, check=True)

    except Exception as e:
        logging.error(f"Error during video processing: {e}")

    finally:
        cap.release()
        logging.info("Cleaning up temporary files.")
        # Remove temporary frame files
        for file_name in os.listdir(temp_folder):
            file_path = os.path.join(temp_folder, file_name)
            os.remove(file_path)
        os.rmdir(temp_folder)
        logging.info("Processing video completed.")

In [18]:
# Process the input video and save the output in MP4 format
input_video_path = r'C:\Users\91748\Downloads\happy.mp4'  # Replace with the actual path
output_video_path = r'C:\Users\91748\Downloads\lli.mp4'  # Replace with the desired output path

process_video(input_video_path, output_video_path)

INFO:root:Total frames in the video: 36
INFO:root:Processing frame 0/36
INFO:root:Processing frame 1/36
INFO:root:Processing frame 2/36
INFO:root:Processing frame 3/36
INFO:root:Processing frame 4/36
INFO:root:Processing frame 5/36
INFO:root:Processing frame 6/36
INFO:root:Processing frame 7/36
INFO:root:Processing frame 8/36
INFO:root:Processing frame 9/36
INFO:root:Processing frame 10/36
INFO:root:Processing frame 11/36
INFO:root:Processing frame 12/36
INFO:root:Processing frame 13/36
INFO:root:Processing frame 14/36
INFO:root:Processing frame 15/36
INFO:root:Processing frame 16/36
INFO:root:Processing frame 17/36
INFO:root:Processing frame 18/36
INFO:root:Processing frame 19/36
INFO:root:Processing frame 20/36
INFO:root:Processing frame 21/36
INFO:root:Processing frame 22/36
INFO:root:Processing frame 23/36
INFO:root:Processing frame 24/36
INFO:root:Processing frame 25/36
INFO:root:Processing frame 26/36
INFO:root:Processing frame 27/36
INFO:root:Processing frame 28/36
INFO:root:Pro